# Task for Today  

***

## Fire/Water Pokémon Classification  

Given *images of Pokémon*, let's try to predict the **type** of a given Pokémon.  
  
We will use a TensorFlow CNN to make our predictions.

# Getting Started

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
from sklearn.preprocessing import minmax_scale

import os
from pathlib import Path
import re

import tensorflow as tf

In [ ]:
train_dir = '../input/pokemon-images-and-types/images/images'
train_path = Path(train_dir)

In [ ]:
files = list(train_path.glob('*.png'))
names = [os.path.split(x)[1] for x in list(train_path.glob('*.png'))]

image_df = pd.concat([pd.Series(names, name='Name'), pd.Series(files, name='Filepath').astype(str)], axis=1)
image_df['Name'] = image_df['Name'].apply(lambda x: re.sub(r'\.\w+$', '', x))
image_df

In [ ]:
label_df = pd.read_csv('../input/pokemon-images-and-types/pokemon.csv')
label_df

In [ ]:
# Merging dfs
train_df = image_df.merge(label_df, on='Name')
train_df = train_df.drop(['Name', 'Type2'], axis=1)
train_df

In [ ]:
# Limiting data to Fire and Water types
train_df = train_df.query("Type1 == 'Fire' | Type1 == 'Water'")
train_df

# Load Image Data

In [ ]:
train_gen = tf.keras.preprocessing.image.ImageDataGenerator(
    validation_split=0.2,
    rescale=1./255
)

In [ ]:
train_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Type1',
    target_size=(120, 120),
    color_mode='rgba',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=1,
    subset='training'
)

val_data = train_gen.flow_from_dataframe(
    train_df,
    x_col='Filepath',
    y_col='Type1',
    target_size=(120, 120),
    color_mode='rgba',
    class_mode='sparse',
    batch_size=32,
    shuffle=True,
    seed=1,
    subset='validation'
)

In [ ]:
image_sample = train_data.next()[0]

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.imshow(image_sample[i, :, :, :])
    plt.axis('off')
plt.show()

# Feature Extraction

In [ ]:
inputs = tf.keras.Input(shape=(120, 120, 4))

conv1 = tf.keras.layers.Conv2D(filters=64, kernel_size=(8, 8), activation='relu')(inputs)
pool1 = tf.keras.layers.MaxPool2D()(conv1)

conv2 = tf.keras.layers.Conv2D(filters=128, kernel_size=(8, 8), activation='relu')(pool1)
pool2 = tf.keras.layers.MaxPool2D()(conv2)

conv3 = tf.keras.layers.Conv2D(filters=256, kernel_size=(8, 8), activation='relu')(pool2)
pool3 = tf.keras.layers.MaxPool2D()(conv3)

outputs = tf.keras.layers.GlobalAveragePooling2D()(pool3)


feature_extractor = tf.keras.Model(inputs=inputs, outputs=outputs)

In [ ]:
feature_extractor.summary()

# Classification

In [ ]:
clf_inputs = feature_extractor.input
clf_outputs = tf.keras.layers.Dense(units=1, activation='sigmoid')(feature_extractor.output)

classifier = tf.keras.Model(inputs=clf_inputs, outputs=clf_outputs)

In [ ]:
classifier.summary()

# Training

In [ ]:
classifier.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

history = classifier.fit(
    train_data,
    validation_data=val_data,
    batch_size=32,
    epochs=100,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=3,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau()
    ]
)

# Visualizing Extracted Features

In [ ]:
feature_extractor.summary()

In [ ]:
feature_extractor.layers

In [ ]:
feature_extractor.layers[1].weights[0].shape

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(64):
    plt.subplot(8, 8, i + 1)
    img = feature_extractor.layers[1].weights[0][:, :, :, i].numpy().copy()
    for channel in range(4):
        img[:, :, channel] = minmax_scale(img[:, :, channel])
    plt.imshow(img)
    plt.axis('off')
plt.show()

# Data Every Day  

This notebook is featured on Data Every Day, a YouTube series where I train models on a new dataset each day.  

***

Check it out!  
https://youtu.be/ioNBmzKJiUE